In [ ]:
!pip install transformers sentencepiece

In [ ]:
# To scrape Wikipedia
from bs4 import BeautifulSoup
# To access contents from URLs
import requests
# Importar el modelo
import transformers
from transformers import pipeline
from transformers import AutoTokenizer

import sentencepiece

import re

# https://www.kaggle.com/code/rajkumarl/wiki-ir-chatbot/notebook

In [ ]:
# Modelo
# modelo = pipeline("question-answering", model="deepset/roberta-base-squad2")

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
from optimum.onnxruntime import ORTModelForQuestionAnswering

model = ORTModelForQuestionAnswering.from_pretrained("optimum/roberta-base-squad2") # onnx checkpoint
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")

modelo = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [ ]:
# Traductores
modelo_en_es= pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
modelo_es_en= pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/802k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/826k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/826k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/802k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

In [ ]:
def traductor_en_es(statement):

  state_traducido = modelo_en_es(statement)
  return state_traducido[0]['translation_text']

def traductor_es_en(statement):

  state_traducido = modelo_es_en(statement)
  return state_traducido[0]['translation_text']

In [ ]:
def procesamiento_tema(tema): 

  # Tema del ChatBot
  # tema = tema.lower().strip().title().replace(' ', '_')

  tema = tema.strip().replace(' ', '_')

  # Link de Wikipedia
  link = 'https://en.wikipedia.org/wiki/'+ tema

  # Información del link de wikipedia
  url_contenido= requests.get(link).content
  soup_tema = BeautifulSoup(url_contenido, 'html.parser')

  # Encontrar solo el texto de wikipedia
  p_texto = soup_tema.findAll('p')

  # Concatenar en texto
  texto = ''

  for t in p_texto:

    rm_style = t.findAll('style')

    for m in rm_style:
      m.decompose()

    t = t.stripped_strings
    t = ' '.join(list(t))
    t = t.replace(r'( ','(').replace(' )', ')').replace(' .', '.').replace(' ,', ',')
    t = re.sub(u'\[.*\u200b', '', t)

    texto = texto + ' '+ t

  # últimos reemplazos en texto

  replace = [(r'[;»«"]', ''),
            (r' \[\d{1,}\]', ''), 
            (r' \[[a-z]{1,}\]', ''),  
            (r'\s+',' '), 
            (r'\(GMT.*\)', ''),
            (r'^\s', '')]

  for old, new in replace:
      texto = re.sub(old, new, texto) 

  return texto

In [ ]:
def modelo_bert(pregunta, texto):

  respuesta = modelo(question = pregunta, context= texto)
  respuesta = respuesta['answer']

  return respuesta 

In [ ]:
def chat():
  
  print('Bienvenido a ChatBot')
  tema = usuario = input('¿Qué tema te gustaría consultar? (Escribe salir si quieres cerrar el chat)')

  if 'salir' not in tema.lower():

    texto = procesamiento_tema(tema)
    usuario = input('¿Qué te gustaría consultar?')

  while 'salir' not in usuario.lower():
    
    respuesta = modelo_bert(usuario, texto)
    print('Wikipedia_Chat: ' + respuesta)
    usuario = input('Usuario:')   


In [ ]:
chat()

In [ ]:
chat()

Bienvenido a ChatBot
¿Qué tema te gustaría consultar? (Escribe salir si quieres cerrar el chat)The Office (American season 1)
¿Qué te gustaría consultar?Who is the main character?
Wikipedia_Chat: Michael Scott
Usuario:When was the show premiere?
Wikipedia_Chat: March 24, 2005
Usuario: What country is the series from?
Wikipedia_Chat: American
Usuario:what is the name of the first episode?
Wikipedia_Chat: Pilot
Usuario:Who is JIm Halpert?
Wikipedia_Chat: sales rep
Usuario:What days did the episodes air?
Wikipedia_Chat: Tuesdays
Usuario:who are the writers of the series?
Wikipedia_Chat: Daniels, Gervais, Merchant, and Michael Schur
Usuario:How many episodes does the season 1 have?
Wikipedia_Chat: six
Usuario:What is the Jim Halpert's job?
Wikipedia_Chat: sales rep
Usuario:What does Pam Beesly?
Wikipedia_Chat: receptionist
Usuario:salir
